In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import gzip
import json
import os
import requests
from io import BytesIO  # Import BytesIO from the io module

In [ ]:
# Define the http adress where the files are recorded.
git_path = 'https://github.com/gunony/ML_Book_Valuations/raw/main/datasets/'

## PART 1 - CONSTITUTION OF THE DATASET BEFORE ANALYSIS AND MACHINE LEARNING
As explained in the readme file, the aim of the project is to predict the rating that would be given to a book based on a certain amount of input data. This first part consists of preparing the data and creating the dataset to be used for the ML algorithm, which constitutes the second part of the project.

### STEP 1 - LOAD THE FILE
As first dataset called 'books.csv' is a curation of Goodreads books. 

In [ ]:
# Download the initial dataset of Books list
url = git_path+'books.csv'
df_Books = pd.read_csv(url,sep=",")

In [ ]:
df_Books.info()

### Step 2 : Adding book's genre in the data frame Books

The genre of the book (history, novel, poetry, etc.) is a factor that readers take into account when making their choice. We can assume that a philosophy book will be read less than a Harry Potter book. This popularity should have an impact on the rating given by readers.

The genre is not included in the initial file. The objective of this step is to therefore add the genre of each books. Goodreads determines a book's genre by crowd-sourcing user shelves. If a number of users shelve a book as "science," for example, then that genre is assigned to the book in their algorithm. This isn't a perfect system, as sometimes users might shelve something as "science" when it's actually "science fiction," and so on. For more details : https://help.goodreads.com/s/article/How-can-I-set-my-book-s-genres. 

The genres of book have been grouped into 10 categories with the score given by each reader. These categories and their respective scores will be added to the initial dataset.

The data set related to book’s genres comes from the great work made by Mengting Wan and Julian McAuley. 
• Mengting Wan, Julian McAuley, "Item Recommendation on Monotonic Behavior Chains", in RecSys'18. [bibtex] 
• Mengting Wan, Rishabh Misra, Ndapa Nakashole, Julian McAuley, "Fine-Grained Spoiler Detection from Large-Scale Review Corpora", in ACL'19. [bibtex]

All files can be found at this adress https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/

In [ ]:
# Function to load the gz file and transform into JSON file 
def load_data(url):
    response = requests.get(url)
    with gzip.GzipFile(fileobj=BytesIO(response.content), mode='rb') as fin:
        data = [json.loads(line.decode('utf-8')) for line in fin]
    return data

# Assuming the function is defined
URL = 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/'
books = load_data(os.path.join(URL, 'goodreads_book_genres_initial.json.gz'))

In [ ]:
# Transform the nested JSON data structure books into a dataframe called df_Genre
df_Genre = pd.json_normalize(books)

# Rename the column with book number - will be necessary for the next steps
df_Genre.rename(columns={'book_id': 'bookID'}, inplace=True)

In [ ]:
display(df_Genre)
df_Genre.info()

#### Remark:
For each book was assigned a score for each of the 10 literary genres. The book's genre with the highest score will be recorded in a specific column. For books without genre the code genres.missing will be given. The NaN will be replaced by 0.

In [ ]:
# Replace NaN with 0 in the dataframe Genre
df_Genre= df_Genre.fillna(0)
# Define a function to get the title of the column with the highest note
def get_highest_note_column(row):
    max_note = row[['genres.history, historical fiction, biography', 'genres.fiction', 'genres.fantasy, paranormal',
                    'genres.mystery, thriller, crime', 'genres.poetry', 'genres.romance', 'genres.non-fiction',
                    'genres.children', 'genres.young-adult', 'genres.comics, graphic']].max()
    return row.index[row == max_note][0]
# Add a new column 'book_genre' to the dataframe
df_Genre['main_genre'] = df_Genre.apply(get_highest_note_column, axis=1)
# Replace 0 with 'genres.missing' in the 'book_genre' column
df_Genre['main_genre'] = df_Genre['main_genre'].replace(0, 'genres.missing')
# Look at the updated dataframe
display(df_Genre)
df_Genre.info()

In [ ]:
# Now we're going to include the data relating to book's genres in the books dataset.

# Convert the bookID column in dataframe Genre from object to int64
df_Genre['bookID'] = df_Genre['bookID'].astype('int64')
# Merge both DataFrames based on the book ID
df_Books = pd.merge(df_Books, df_Genre, left_on='bookID', right_on='bookID', how='left')

In [ ]:
# check the final result of the books dataframe after adding book's genre.
df_Books.info()

#### Remarks :
In the books dataframe there 14 485 books. Only 12664 books have a book's genre. The remaining 1821 books don't have one. 
We will replace the NaN of these 1821 books with the number 0.

How to explain the absence of this information, there are two possibilities. Either these books are new and readers have not had time to read or categorise them. Or these books have simply never been read. This information will be confirmed by the book's publication date.

For the rest of the project, we are going to replace the 'NaN' of these 1824 books with the number zero. We will still be able to find these books by summing the scores of the 10 columns representing the book genres. Or by selecting the 'main_genre' column if there is no result. 

### Step 3 - Calculate the seniority of the books
From the Publication date until today.

In [ ]:
from datetime import datetime

# Check if column publication_date is correct 
for index, row in df_Books.iterrows():
    try:
        date_obj = datetime.strptime(row['publication_date'], '%m/%d/%Y')
    except ValueError as e:
        print(f"Error in row {index}: {e}")
        print("Row details:", row)

In [ ]:
# Convert 'publication_date' to datetime with US format
df_Books['publication_date'] = pd.to_datetime(df_Books['publication_date'], format='%m/%d/%Y')

In [ ]:
# Calculate the number of months from the Publication date until today
today = datetime.today()
df_Books['Months_Until_Today'] = (today.year - df_Books['publication_date'].dt.year)*12+(today.month - df_Books['publication_date'].dt.month)

In [ ]:
df_Books.info()

### Step 4 : Book's language

In [ ]:
df_Books['language_code'].value_counts()

#### Remarks : 
There are miscellaneous languages. But english is the most representative as Good reads is a US web site.
We are going to group English-language books under a single code eng.

In [ ]:
df_Books['language_code'].replace(['en-CA', 'en-GB', 'en-US', 'enm'],'eng', inplace=True)

### Step 5 : Work on the authors
The authors also have a rating which we will retrieve and add to the books dataset.
In the authors column of the Books dataset, some books have several names. The first corresponds to the writer (e.g. Harry Potter: 'J.K. Rowling/Mary GrandPrÃ©'). The other names may be illustrators (in the case of Mary GrandPrÃ©), translators or co-authors. 
Unfortunately, this is information is not available. For the rest of the project, we will therefore keep only the first name that corresponds to the author or one of the authors. This decision was validated after testing a few books.
Information on the authors is available on the following site: https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/

In [ ]:
# Split the author column to keep only the name of the writer
df_Books[['authors','co-authors']] = df_Books['authors'].str.split(pat='/', n=1, expand=True)

In [ ]:
# Function to load the gz file and transform into JSON file 
def load_data(url):
    response = requests.get(url)
    with gzip.GzipFile(fileobj=BytesIO(response.content), mode='rb') as fin:
        data = [json.loads(line.decode('utf-8')) for line in fin]
    return data

# Assuming the function is defined
URL = 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/'
books = load_data(os.path.join(URL, 'goodreads_book_authors.json.gz'))

In [ ]:
# Transform the nested JSON data structure books into a dataframe called df_Genre
df_Authors = pd.json_normalize(books)

In [ ]:
# A look at the dataframe
df_Authors.info()

In [ ]:
# Create a mapping dictionnary containing only the columns name and average_rating
auteur_map = dict(zip(df_Authors['name'], df_Authors['average_rating']))

# Adding Authors rating to df-Books
df_Books['author_rating'] = df_Books['authors'].map(auteur_map)

# For the authors not in the csv file, we give a note of zero.
df_Books['author_rating'] = df_Books['author_rating'].fillna(0)

In [ ]:
df_Books.info()

In [ ]:
# change Dtype of author rating
df_Books['author_rating'] = df_Books['author_rating'].astype('float64')

In [ ]:
# check if there is NaN in the column author rating
Has_Nan = df_Books['author_rating'].isnull().sum()
print(Has_Nan)

### Step 6 : Adding format of the books
We add the format of the book (paper, audio,...) from the the file 'goodreads_books.json.gz' which is located on the web site : https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/

As the file is large (1.9 GB), it was uploaded and converted into an easier-to-load csv file separately containing only the prinicpal information needed : bookID + Format + isebook. 

In [ ]:
# Download the Book's format csv file
df_BooksFormat = pd.read_csv(r"C:\Users\gunon\Documents\bootcamp-main\3-projects\ML_Book_Valuations\datasets\BooksFormat.csv", sep=",")
# What is inside the file
df_BooksFormat.info()

In [ ]:
# solve the alert message above
df_BooksFormat['format'] = df_BooksFormat['format'].astype (object)

In [ ]:
df_BooksFormat.info()

In [ ]:
# Create a mapping dictionary containing only BookID and Format from the file BooksList.csv
book_format = dict(zip(df_BooksFormat['book_id'], df_BooksFormat['format']))

# Add format to the Books dataframe using the map function
df_Books['book_format'] = df_Books['bookID'].map(book_format)

# Group book's format into two families: audio / books
df_Books['book_format'] = df_Books['book_format'].fillna('Unknown Binding')
df_Books['book_format'] = df_Books.apply(lambda row: 'audio' if 'audio' in row['publisher'].lower() and
                        row['book_format'] == 'Unknown Binding' else row['book_format'], axis=1)
df_Books['book_format'] = df_Books['book_format'].replace(['Audio', 'Audio CD', 'audio Cassette', 'Audio Cassette'
                        ,'Audiobook', 'MP3 CD'], 'audio')
df_Books['book_format'] = df_Books['book_format'].replace(['Unknown Binding'], 'book')
df_Books['book_format'] = df_Books['book_format'].apply(lambda x: 'book' if x != 'audio' else x)

# get_dummies tranforms categorical data to numbers 0=audio / 1=book
df_Books["audio=0_book=1"] = pd.get_dummies(df_Books.book_format, drop_first=True).values

In [ ]:
df_Books.info()

In [ ]:
# check the new columns
df_Books['book_format'].value_counts()

In [ ]:
df_Books['audio=0_book=1'].value_counts()

In [ ]:
### Step 7 : Save the final file

In [ ]:
# Saving Books dataframe with all data on github
df_Books.to_csv(r"C:\Users\gunon\Documents\bootcamp-main\3-projects\ML_Book_Valuations\datasets\Books_Final.csv", index=False)